In [1]:
# Set Variables
SW_LAT = -38.88917
SW_LON = 140.84444
NE_LAT = -33.70833
NE_LON = 150.0

PARAMETER_NAME = "Pat1_C_B_1_DailyMean"
PROPERTY_NAME = "Turbidity"
TIME_PERIOD = "2010/2024"


In [2]:
import requests
from xml.etree.ElementTree import Element, SubElement, tostring

# Define the namespace map to use in the XML
ns = {
    'sos': "http://www.opengis.net/sos/2.0",
    'fes': "http://www.opengis.net/fes/2.0",
    'gml': "http://www.opengis.net/gml/3.2",
    'xsi': "http://www.w3.org/2001/XMLSchema-instance",
    'sams': "http://www.opengis.net/samplingSpatial/2.0"
}

# Define the endpoint URL for the SOS service
url = "http://www.bom.gov.au/waterdata/services"

# Create XML elements for the request
envelope = Element('sos:GetFeatureOfInterest', {
    'service': 'SOS', 'version': '2.0', 'xmlns:sos': ns['sos'], 'xmlns:fes': ns['fes'], 'xmlns:gml': ns['gml'], 'xmlns:xsi': ns['xsi'], 'xmlns:sams': ns['sams']
})

spatial_filter = SubElement(envelope, 'sos:spatialFilter')
bbox = SubElement(spatial_filter, 'fes:BBOX')
value_ref = SubElement(bbox, 'fes:ValueReference')
value_ref.text = 'om:featureOfInterest/*/sams:shape'

envelope_ = SubElement(bbox, 'gml:Envelope', {
    'srsName': 'urn:ogc:def:crs:EPSG::4326'
})
lower_corner = SubElement(envelope_, 'gml:lowerCorner')
upper_corner = SubElement(envelope_, 'gml:upperCorner')

# Insert the actual coordinates for the Murray River bounding box
lower_corner.text = f"{SW_LAT} {SW_LON}"  # Southernmost, westernmost
upper_corner.text = f"{NE_LAT} {NE_LON}"  # Northernmost, easternmost

# Add observed property element
observed_property = SubElement(envelope, 'sos:observedProperty')
observed_property.text = f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}"  # Example observed property


# Convert the ElementTree to a string
request_xml = tostring(envelope, 'utf-8')

# Define headers for the POST request
headers = {
    'Content-Type': 'application/xml'
}

# Send the POST request
FeatureOfInterestResponse = requests.post(url, data=request_xml, headers=headers)

# Print the response from the server


In [3]:
import requests
import xml.etree.ElementTree as ET
# Sample XML response from GetFeatureOfInterest
xml_response = FeatureOfInterestResponse.text

# Parse the XML string
root = ET.fromstring(xml_response)

# Extract station IDs
station_ids = [elem.text.split('/')[-1] for elem in root.findall('.//gml:identifier', namespaces={'gml': 'http://www.opengis.net/gml/3.2'})]

print(station_ids) 

#get stations within region


['WYS00BSN', '405209', '41000271', 'WARMUZ01', 'WARMDZ01', 'WAREUV01', 'WARMWEIR', 'WARWUV01', '408203', '225234', '410167', '233217', '233224', '233200', '233247', '233269', '233201', '227231', '219026', '219003', '221212', '221225', '405264', '405227', '410134', 'WBITTEAO', 'WBITBASO', 'WBITTWEO', 'WBMRCANN', 'WBMRBESS', 'WBMRGARF', 'WBMRPAKN', 'WBMRTYNG', 'WBRAHAMS', '41000270', '219025', '404216', '403233', '403222', '403254', 'WBULLKFR', '228213', 'WBUNDIVW', 'WBUNTTPD', 'WBUNRDSD', 'WBUNRUSD', 'WBUNWEIR', '410717', '340013', '410774', '406207', '406213', '221224', '228228', 'WCARCKDS', 'WCARCKUS', 'WCADNT15', 'WCADNT05', 'WCADNT70', 'WCADNT65', 'WCADNT60', 'WCADNT58', 'WCADNT55', 'WCADNT50', 'WCADNT40', 'WCADNT35', 'WCADNT30', 'WCADNT25', 'WCADNT20', 'WCADNT10', 'WCADBOTT', 'WCADNT00', 'WCADRE01', 'WCADRE02', 'WCARTPOL', 'WTHOMCAC', 'WCEMUZ01', 'WCMEUZ01', 'WCMWAV01', '406215', '221211', 'WCONDR22', 'WCONTR22', '410081', 'WCORAV01', 'WCORIF01', '401230', '401229', '410733', '4107

In [4]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

# Define the SOS2 endpoint URL
url = "http://www.bom.gov.au/waterdata/services"

data = []  # List to store data for DataFrame


# Define the parameters for the GetObservation request
params = {
    'service': 'SOS',
    'version': '2.0.0',
    'request': 'GetObservation',
    'procedure': f"http://bom.gov.au/waterdata/services/tstypes/{PARAMETER_NAME}",
    'observedProperty': f"http://bom.gov.au/waterdata/services/parameters/{PROPERTY_NAME}",
    'temporalFilter': f"om:phenomenonTime,{TIME_PERIOD}"
}

# Assuming station_ids is defined earlier in the script
for station_id in station_ids:
    print(f"Querying data for station: {station_id}")
    
    # Add the station ID to the parameters
    params['featureOfInterest'] = f"http://bom.gov.au/waterdata/services/stations/{station_id}"
    
    # Make the GetObservation request to the SOS2 endpoint
    response = requests.get(url, params=params, timeout=100)  # Added timeout
    
    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        
        # Find all the MeasurementTVP elements which contain the time-value pairs
        tvp_elements = root.findall('.//{http://www.opengis.net/waterml/2.0}MeasurementTVP')
        
        if not tvp_elements:
            print("No data found for this station.")
        else:
            # Print out the time and value for each MeasurementTVP
            for tvp in tvp_elements:
                time = tvp.find('{http://www.opengis.net/waterml/2.0}time').text
                value = tvp.find('{http://www.opengis.net/waterml/2.0}value').text
                #print(f"  {time}: {value}")
                data.append({'Station': station_id, 'Time': time, 'Value': value})
    else:
        print(f"Failed to retrieve data for station {station_id}. HTTP status code: {response.status_code}")
        print("Response Headers:", response.headers)
        print("Response Body:", response.text)


df = pd.DataFrame(data)

pivot_df = df.pivot_table(index='Time', values='Value', columns='Station', aggfunc='first')

# Show pivoted DataFrame
print(pivot_df)
# Show DataFrame

Querying data for station: WYS00BSN
No data found for this station.
Querying data for station: 405209
Querying data for station: 41000271
Querying data for station: WARMUZ01
Querying data for station: WARMDZ01
No data found for this station.
Querying data for station: WAREUV01
No data found for this station.
Querying data for station: WARMWEIR
Querying data for station: WARWUV01
No data found for this station.
Querying data for station: 408203
Querying data for station: 225234
Querying data for station: 410167
No data found for this station.
Querying data for station: 233217
Querying data for station: 233224
Querying data for station: 233200
Querying data for station: 233247
Querying data for station: 233269
Querying data for station: 233201
Querying data for station: 227231
Querying data for station: 219026
No data found for this station.
Querying data for station: 219003
No data found for this station.
Querying data for station: 221212
Querying data for station: 221225
Querying data 

In [5]:
pivot_df.to_csv('DailyMeanTurbidity.csv', index=True)